# 태스크 1a: 텍스트 생성 수행

이 노트북에서는 대규모 언어 모델(LLM)을 사용하여 지원 엔지니어로부터 받은 고객 서비스의 품질에 대해 부정적인 피드백을 제공한 고객에게 보내는 이메일 응답을 생성하는 방법을 알아봅니다. 이 노트북에서는 고객의 이전 이메일을 기반으로 감사의 메시지가 포함된 이메일을 생성합니다. Boto3 클라이언트와 함께 Amazon Bedrock API를 사용하여 Amazon Titan 모델을 사용합니다.

이 태스크에서 사용하는 프롬프트를 zero-shot 프롬프트라고 합니다. 이 zero-shot 프롬프트에서는 언어 모델에 대한 태스크 또는 원하는 출력을 일반 언어로 설명합니다. 그러면 모델에서는 사전 훈련된 지식과 기능을 사용하여 제공된 프롬프트만을 기반으로 응답을 생성하거나 태스크를 완료합니다.

#### 시나리오
당신은 AnyCompany의 고객 서비스 관리자인 Bob이며, 일부 고객이 고객 서비스에 만족하지 못하고 고객 지원 엔지니어가 제공하는 서비스에 대한 부정적인 피드백을 제공하고 있습니다. 이제 이러한 고객에게 서비스 불만에 대해 사과하고, 신뢰를 회복하기 위해 대응하려고 합니다. 이전 이메일 기반으로 고객의 정서에 맞게 인간 친화적이고 개인화된 대량의 이메일을 생성하려면 대규모 언어 모델(LLM)의 도움이 필요합니다.

## 태스크 1a.1: 환경 설정

이 태스크에서는 환경을 설정합니다.

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## 태스크 1a.2: 텍스트 생성

이 태스크에서는 Amazon Bedrock 서비스를 통해 이메일을 생성하는 입력을 준비합니다.

In [ ]:
# create the prompt
prompt_data = """
Command: Write an email from Bob, Customer Service Manager, AnyCompany to the customer "John Doe" 
who provided negative feedback on the service provided by our customer support 
engineer"""

In [ ]:
body = json.dumps({
    "inputText": prompt_data, 
    "textGenerationConfig":{
        "maxTokenCount":8192,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

그런 다음에 Amazon Titan 모델을 사용합니다.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **참고:** Amazon Titan에서는 4k 이하 토큰의 컨텍스트 창이 지원하며 다음과 같은 파라미터를 허용합니다.
- `inputText`: LLM에 대한 프롬프트
- `textGenerationConfig`: 출력을 생성하는 동안 모델이 고려할 파라미터

Amazon Bedrock API는 다음을 허용하는 API `invoke_model`을 제공합니다.
- `modelId`: Amazon Bedrock에서 사용할 수 있는 다양한 파운데이션 모델의 고유 식별자
- `accept`: 입력 요청의 유형
- `contentType`: 출력의 콘텐츠 유형
- `body`: 프롬프트 및 구성으로 이루어진 JSON 문자열

사용 가능한 텍스트 생성 모델 ID는 [설명서](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html)를 참조하십시오.

## 태스크 1a.3: Amazon Titan Large 언어 모델 호출

이 태스크에서는 모델이 이전에 생성한 프롬프트를 토대로 출력을 생성하는 방식을 알아봅니다.

### 전체 출력 생성

이 이메일은 입력 요청을 이해하고 다양한 양식에 대한 고유한 이해를 이용하여 Amazon Titan Large 모델을 통해 생성됩니다. API에 대한 요청은 동기식이며, 모델에서 전체 출력이 생성될 때까지 기다립니다.

In [ ]:
#invoke model
modelId = 'amazon.titan-text-express-v1' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"
try:

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('results')[0].get('outputText')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error


In [ ]:
# The relevant portion of the response begins after the first newline character
# Below we print the response beginning after the first occurence of '\n'.

email = outputText[outputText.index('\n')+1:]
print(email)


### 출력 생성 스트리밍

또한 Bedrock은 청크 형식으로 모델이 생성한 출력을 스트리밍할 수 있도록 지원합니다. 이 이메일은 스트리밍 옵션으로 모델을 호출하여 생성됩니다. `invoke_model_with_response_stream`에서는 사용자가 읽을 수 있는 `ResponseStream`을 반환됩니다.

In [ ]:
# invoke model with response stream
output = []
try:
    
    response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
    stream = response.get('body')
    
    i = 1
    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['outputText']
                output.append(text)
                print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
                i+=1
            
except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

응답 포함 스트리밍 접근 방식은 모델의 출력을 빠르게 가져오고, 사용자가 출력을 읽을 때 서비스가 출력을 완료할 수 있도록 하는 데 도움이 됩니다. 이는 더 긴 텍스트 조각을 생성하도록 모델에 요청하는 사용 사례에 도움이 됩니다. 생성된 청크를 나중에 모두 조합하여 완전한 출력을 형성하고 사용 사례에 사용할 수 있습니다. 

In [ ]:
#combine output chunks
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(complete_output)


지금까지 Amazon Bedrock API에 기본 노출을 제공하는 boto3 SDK를 사용하여 실험해 보았습니다. 이 API를 사용하여 고객의 부정적인 피드백에 응답할 이메일을 생성하는 사용 사례를 살펴보았습니다.

### 직접 해보기
- 프롬프트를 특정 사용 사례로 변경하고 다양한 모델의 출력을 평가하십시오.
- 토큰 길이를 조작하면서 서비스의 지연 시간 및 응답성을 이해해 보십시오.
- 다양한 프롬프트 엔지니어링 원칙을 적용하여 더 나은 출력을 얻을 수 있습니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 이 노트북 파일을 닫고 **Task1b_ko_kr.ipynb**를 계속합니다.